<a href="https://colab.research.google.com/github/C241-PS517-ISALAT/Isalat-Model/blob/model_test_2/Datapreproc_Model2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, optimizers, regularizers

import os
import shutil

In [3]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
train_data = "/content/drive/My Drive/bisindo/data/train"
augmented_images_dir = "/content/drive/My Drive/bisindo/data/augmented_images"
val_data = "/content/drive/My Drive/bisindo/data/validation"

In [11]:
# Ensure the augmented images directory exists
os.makedirs(augmented_images_dir, exist_ok=True)


In [12]:
# Define image dimensions
img_width, img_height = 227, 227

# Create ImageDataGenerator for generating augmented images
datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=[0.8, 1.2]
)

# Load images from directory and generate augmented images
for class_dir in os.listdir(train_data):
    class_path = os.path.join(train_data, class_dir)
    if not os.path.isdir(class_path):
        continue

    save_dir = os.path.join(augmented_images_dir, class_dir)
    os.makedirs(save_dir, exist_ok=True)

    generator = datagen.flow_from_directory(
        train_data,
        classes=[class_dir],
        target_size=(img_width, img_height),
        batch_size=1,  # Generate one image at a time
        class_mode=None,
        save_to_dir=save_dir,
        save_prefix='aug',
        save_format='jpeg'
    )

    # Generate 50 augmented images per class
    for i in range(100):
        generator.next()

Found 9 images belonging to 1 classes.
Found 9 images belonging to 1 classes.
Found 9 images belonging to 1 classes.
Found 9 images belonging to 1 classes.
Found 9 images belonging to 1 classes.
Found 9 images belonging to 1 classes.
Found 9 images belonging to 1 classes.
Found 9 images belonging to 1 classes.
Found 9 images belonging to 1 classes.
Found 9 images belonging to 1 classes.
Found 9 images belonging to 1 classes.
Found 9 images belonging to 1 classes.
Found 9 images belonging to 1 classes.
Found 9 images belonging to 1 classes.
Found 9 images belonging to 1 classes.


KeyboardInterrupt: 

In [7]:
# Define new directories for training including augmented images
augmented_train_data = "/content/drive/My Drive/bisindo/data/augmented_and_train"

# Ensure the directory exists
os.makedirs(augmented_train_data, exist_ok=True)

# Move original and augmented images to a new combined directory

# Move original images
for class_dir in os.listdir(train_data):
    source_dir = os.path.join(train_data, class_dir)
    dest_dir = os.path.join(augmented_train_data, class_dir)
    os.makedirs(dest_dir, exist_ok=True)
    for file_name in os.listdir(source_dir):
        shutil.copy(os.path.join(source_dir, file_name), dest_dir)

# Move augmented images
for class_dir in os.listdir(augmented_images_dir):
    source_dir = os.path.join(augmented_images_dir, class_dir)
    dest_dir = os.path.join(augmented_train_data, class_dir)
    os.makedirs(dest_dir, exist_ok=True)
    for file_name in os.listdir(source_dir):
        shutil.move(os.path.join(source_dir, file_name), dest_dir)


In [8]:
# Create ImageDataGenerator for training and validation data
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

# Load training data
train_generator = train_datagen.flow_from_directory(
    augmented_train_data,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='sparse'
)

# Load validation data
validation_generator = val_datagen.flow_from_directory(
    val_data,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='sparse'
)

Found 754 images belonging to 26 classes.
Found 78 images belonging to 26 classes.


In [9]:
# Define SqueezeNet architecture based on the diagram
def fire_module(x, squeeze_filters, expand_filters):
    x = layers.Conv2D(squeeze_filters, (1, 1), padding='valid', activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
    left = layers.Conv2D(expand_filters, (1, 1), padding='valid', activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
    right = layers.Conv2D(expand_filters, (3, 3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.concatenate([left, right])
    return x

input_shape = (img_width, img_height, 3)
input_img = layers.Input(shape=input_shape)

x = layers.Conv2D(64, (3, 3), strides=(2, 2), padding='valid', activation='relu', kernel_regularizer=regularizers.l2(0.001))(input_img)
x = layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)
x = fire_module(x, squeeze_filters=16, expand_filters=64)
x = fire_module(x, squeeze_filters=16, expand_filters=64)
x = layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)
x = fire_module(x, squeeze_filters=32, expand_filters=128)
x = fire_module(x, squeeze_filters=32, expand_filters=128)
x = layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)
x = fire_module(x, squeeze_filters=48, expand_filters=192)
x = fire_module(x, squeeze_filters=48, expand_filters=192)
x = fire_module(x, squeeze_filters=64, expand_filters=256)
x = fire_module(x, squeeze_filters=64, expand_filters=256)
x = layers.Dropout(0.5)(x)
x = layers.Conv2D(1000, (1, 1), padding='valid', activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
x = layers.GlobalAveragePooling2D()(x)
output = layers.Activation('softmax')(x)

model = models.Model(input_img, output)


Epoch 1/20
24/24 [==============================] - 148s 6s/step - loss: 9.0734 - accuracy: 0.0332 - val_loss: 8.9718 - val_accuracy: 0.0385
Epoch 2/20
24/24 [==============================] - 101s 4s/step - loss: 8.8812 - accuracy: 0.0385 - val_loss: 8.7830 - val_accuracy: 0.0385
Epoch 3/20
24/24 [==============================] - 92s 4s/step - loss: 8.6290 - accuracy: 0.0385 - val_loss: 8.2161 - val_accuracy: 0.0385
Epoch 4/20
24/24 [==============================] - 94s 4s/step - loss: 6.4626 - accuracy: 0.0292 - val_loss: 5.3580 - val_accuracy: 0.0385
Epoch 5/20
18/24 [=====================>........] - ETA: 22s - loss: 5.1236 - accuracy: 0.0302

KeyboardInterrupt: 

In [ ]:
# Compile the model with a lower learning rate and additional metrics
model.compile(optimizer='adam', loss='SparseCategoricalCrossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(train_generator, epochs=20, validation_data=validation_generator)